# Notebook I - Cleaning & Processing `bike_trip_data.csv`

---
I am going to start by importing Pandas and reading in both datasets, after which I'll go through each thoroughly and clean/process them.

In [402]:
# imports
import pandas as pd
from datetime import datetime

---
## 1. Cleaning Bike Trips Data

In [334]:
# read trips
trips = pd.read_csv("./data/bike_trip_data.csv")

This data is in **panel** format, meaning that it is a time-series cross section (multiple unique things being tracked across multiple points in time). 

In [335]:
# ensuring the data read in properly
trips.head()

,month,day,year,start_time,duration_minutes,bikeid,start_station_id,start_station_name,end_station_id,end_station_name
0,1,1,2014,0:12,20,428.0,2495.0,4th & Congress,2502.0,Barton Springs & Riverside
1,1,1,2014,0:12,19,14.0,2495.0,4th & Congress,2502.0,Barton Springs & Riverside
2,1,1,2014,0:12,17,172.0,2495.0,4th & Congress,2502.0,Barton Springs & Riverside
3,1,1,2014,0:12,8,417.0,2498.0,Convention Center / 4th St. @ MetroRail,2498.0,Convention Center / 4th St. @ MetroRail
4,1,1,2014,2:12,15,40.0,2496.0,8th & Congress,2503.0,South Congress & James


In [336]:
print(f"Rows:   {trips.shape[0]}")
print(f"Columns:   {trips.shape[1]}")

Rows:   328670
Columns:   10


In [337]:
# quick check on data types
trips.info() 

# we will address the data types and missing values later!

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 328670 entries, 0 to 328669
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   month               328670 non-null  int64  
 1   day                 328670 non-null  int64  
 2   year                328670 non-null  int64  
 3   start_time          328670 non-null  object 
 4   duration_minutes    328670 non-null  int64  
 5   bikeid              328125 non-null  float64
 6   start_station_id    311219 non-null  float64
 7   start_station_name  328670 non-null  object 
 8   end_station_id      310482 non-null  float64
 9   end_station_name    328670 non-null  object 
dtypes: float64(3), int64(4), object(3)
memory usage: 25.1+ MB


---
### 1a. Looking for Potentially Nonsensical Data

Negative trip durations?

In [338]:
# let's make sure there are NO "negative" trip durations logged
trips[trips['duration_minutes']<0]

,month,day,year,start_time,duration_minutes,bikeid,start_station_id,start_station_name,end_station_id,end_station_name


Trips with duration of 0 minutes?

In [339]:
trips[trips['duration_minutes']==0]

,month,day,year,start_time,duration_minutes,bikeid,start_station_id,start_station_name,end_station_id,end_station_name
44,1,1,2014,11:12,0,110.0,2504.0,South Congress & Elizabeth,2504.0,South Congress & Elizabeth
171,1,1,2014,16:12,0,999.0,2498.0,Convention Center / 4th St. @ MetroRail,2498.0,Convention Center / 4th St. @ MetroRail
180,1,1,2014,16:12,0,646.0,2499.0,City Hall / Lavaca & 2nd,2501.0,5th & Bowie
185,1,1,2014,16:12,0,49.0,2501.0,5th & Bowie,2496.0,8th & Congress
193,1,1,2014,16:12,0,252.0,2502.0,Barton Springs & Riverside,2502.0,Barton Springs & Riverside
...,...,...,...,...,...,...,...,...,...,...
327639,12,29,2015,13:12,0,276.0,2568.0,East 11th St. at Victory Grill,2568.0,East 11th St. at Victory Grill
327640,12,29,2015,13:12,0,276.0,2568.0,East 11th St. at Victory Grill,2568.0,East 11th St. at Victory Grill
327655,12,29,2015,14:12,0,27.0,2494.0,2nd & Congress,2494.0,2nd & Congress
328071,12,30,2015,15:12,0,470.0,2494.0,2nd & Congress,2494.0,2nd & Congress


A couple things come to my mind looking at trips with 0 minutes of duration. <br>

(1.) Is it possible to even have a 0-minute trip? <br>
(2.) If a trip was just cancelled, potentially logging as a 0-minute trip, would we want to keep this data?

#### (1.) Is it possible to have a 0-minute trip?
First, we can look into 0-minute trips that have different end stations. Do these make sense?

In [340]:
# filter for trips where start != end & duration = 0
trips[(trips['start_station_id'] != trips['end_station_id']) & (trips['duration_minutes']==0)]

,month,day,year,start_time,duration_minutes,bikeid,start_station_id,start_station_name,end_station_id,end_station_name
180,1,1,2014,16:12,0,646.0,2499.0,City Hall / Lavaca & 2nd,2501.0,5th & Bowie
185,1,1,2014,16:12,0,49.0,2501.0,5th & Bowie,2496.0,8th & Congress
306,1,2,2014,13:12,0,52.0,NaN,Main Office,NaN,Main Office
980,1,11,2014,11:12,0,275.0,2502.0,Barton Springs & Riverside,2501.0,5th & Bowie
1083,1,11,2014,15:12,0,152.0,2494.0,2nd & Congress,2501.0,5th & Bowie
...,...,...,...,...,...,...,...,...,...,...
313594,11,13,2015,17:12,0,132.0,2497.0,Capitol Station / Congress & 11th,3292.0,East 4th & Chicon
313595,11,13,2015,17:12,0,177.0,2497.0,Capitol Station / Congress & 11th,3292.0,East 4th & Chicon
313596,11,13,2015,17:12,0,805.0,2497.0,Capitol Station / Congress & 11th,3292.0,East 4th & Chicon
313660,11,13,2015,19:12,0,414.0,2571.0,Red River & 8th Street,3292.0,East 4th & Chicon


It is totally possible that trips showing as 0 minutes were rounded down since the trip time was less than 1 minute. For example, one could travel a block or two, on an exceptionally hot day, in less than a minute instead of walking. However, I did some gut checks and examined the distance between a few of these stations on Google Maps. For example, [this is the distance between Congress & 11th and East 4th & Chicon.](https://www.google.com/maps/dir/Congress+Ave.+%26+E+11th+St,+Austin,+TX+78701/Chicon+St+%26+E+4th+St,+Austin,+TX+78702/@30.2670388,-97.735376,16z/data=!4m14!4m13!1m5!1m1!1s0x8644b5a085d8fa5f:0x970ba0349dd84ce9!2m2!1d-97.7410821!2d30.2727434!1m5!1m1!1s0x8644b5b427f1531b:0x8af9df67bb00f2d3!2m2!1d-97.7234017!2d30.2597961!3e1?entry=ttu) This is nearly 2 miles, and would take approximately 11 minutes of travel time via bike. In this case, this trip would not make any sense at all. [This is the approximate distance between 5th & Bowie and the corner of Lavaca & 2nd](https://www.google.com/maps/dir/30.2649312,-97.7465533/5th%2FBowie,+Austin,+TX/@30.26645,-97.7486182,17.81z/data=!4m9!4m8!1m0!1m5!1m1!1s0x8644b511ff8d6033:0x843e38b3ca33d105!2m2!1d-97.752626!2d30.269522!3e1?entry=ttu). This trip is shorter, but it would still take roughly 5 minute to complete via bike. <br>
To thoroughly filter through these trips and provide an extra piece of data, it may prove useful to add a `trip_distance` variable that uses a Google Maps API to calculate the distance between start and end stations. Therefore, we could validate the `duration_minutes` of many trips. <br> For now, since there is uncertainty for 0-minute trips, I will mark them as drop candidates.

#### (2.) If a trip was just cancelled, potentially logging as a 0-minute trip, would we want to keep this data?
Now, we can look at 0-minute trips where the start and end stations are the same.

In [341]:
# filter for trips where start==end & duration = 0
trips[(trips['start_station_id']==trips['end_station_id']) & (trips['duration_minutes']==0)]

,month,day,year,start_time,duration_minutes,bikeid,start_station_id,start_station_name,end_station_id,end_station_name
44,1,1,2014,11:12,0,110.0,2504.0,South Congress & Elizabeth,2504.0,South Congress & Elizabeth
171,1,1,2014,16:12,0,999.0,2498.0,Convention Center / 4th St. @ MetroRail,2498.0,Convention Center / 4th St. @ MetroRail
193,1,1,2014,16:12,0,252.0,2502.0,Barton Springs & Riverside,2502.0,Barton Springs & Riverside
223,1,1,2014,17:12,0,446.0,2502.0,Barton Springs & Riverside,2502.0,Barton Springs & Riverside
266,1,1,2014,23:12,0,925.0,2503.0,South Congress & James,2503.0,South Congress & James
...,...,...,...,...,...,...,...,...,...,...
327639,12,29,2015,13:12,0,276.0,2568.0,East 11th St. at Victory Grill,2568.0,East 11th St. at Victory Grill
327640,12,29,2015,13:12,0,276.0,2568.0,East 11th St. at Victory Grill,2568.0,East 11th St. at Victory Grill
327655,12,29,2015,14:12,0,27.0,2494.0,2nd & Congress,2494.0,2nd & Congress
328071,12,30,2015,15:12,0,470.0,2494.0,2nd & Congress,2494.0,2nd & Congress


Let's assume some trips were cancelled immediately for a number of reasons: 
- the rider changed their method of commute 
- the bike was dysfunctional 
- the rider requested a refund <br>

One could argue that we could still keep these trips in our data for revenue purposes since revenue for a single trip is a function of duration in minutes with a 1-dollar base fee: <center>$1 + 0.10*minutes$<center>
<div style="text-align: left">However, without reasonable certainty, I would argue that keeping these trips may affect prediction results. Considering that that these 0-minute trips do not make up a substantial chunk of our overall trip data, I will drop them.</div>

In [342]:
# dropping trips with 0-minutes (filter on all trips > 0 mins)
trips = trips[trips['duration_minutes'] > 0]

trips.shape # rechecking dimensions

(322281, 10)

---
### 1b. Missing Data

In [343]:
trips.isnull().sum().sort_values(ascending = False)

end_station_id        17999
start_station_id      17228
bikeid                  522
month                     0
day                       0
year                      0
start_time                0
duration_minutes          0
start_station_name        0
end_station_name          0
dtype: int64

Missing data for:
- `bikeid`
- `start_station_id`
- `end_station_id`

---
#### Investigating Missing `bikeid` Data

In [344]:
trips[trips['bikeid'].isna() == True]

,month,day,year,start_time,duration_minutes,bikeid,start_station_id,start_station_name,end_station_id,end_station_name
9961,3,6,2014,11:12,41,NaN,2498.0,Convention Center / 4th St. @ MetroRail,2571.0,Red River & 8th Street
10447,3,7,2014,11:12,1,NaN,2542.0,Plaza Saltillo,2542.0,Plaza Saltillo
10635,3,7,2014,13:12,11,NaN,2571.0,Red River & 8th Street,2501.0,5th & Bowie
10844,3,7,2014,16:12,13,NaN,2498.0,Convention Center / 4th St. @ MetroRail,2550.0,Republic Square @ Guadalupe & 4th St.
10858,3,7,2014,16:12,12,NaN,2501.0,5th & Bowie,2498.0,Convention Center / 4th St. @ MetroRail
...,...,...,...,...,...,...,...,...,...,...
297772,10,11,2015,18:12,13,NaN,2574.0,Zilker Park,2501.0,5th & Bowie
297793,10,11,2015,19:12,10,NaN,2501.0,5th & Bowie,2712.0,Toomey Rd @ South Lamar
297810,10,11,2015,19:12,20,NaN,2574.0,Zilker Park,2495.0,4th & Congress
297818,10,11,2015,19:12,13,NaN,2712.0,Toomey Rd @ South Lamar,1006.0,Zilker Park West


For trips missing a `bikeid`, my initial reaction is to question possible reasons `bikeid` could be missing in the first place. A trip cannot be taken without a bike, and assuming an ID is prescribed to each bike, each trip should have a corresponding `bikeid`. <br><br> I want to drop any missing bikeid data. For example, trips with ***missing*** `bikeid` and a duration of 0 minutes or trips that started and ended in the same station could be trips that were cancelled or voided by the bike-share company. Since there are still over 300,000 rows of trip data, it seems reasonable to avoid data we are not entirely certain is real or not, especially because it may skew predictions.

In [345]:
trips[trips['bikeid'].isna() == True].sort_values(by = 'duration_minutes', ascending = True)
# dropping 522 rows

,month,day,year,start_time,duration_minutes,bikeid,start_station_id,start_station_name,end_station_id,end_station_name
15802,3,11,2014,15:12,1,NaN,2498.0,Convention Center / 4th St. @ MetroRail,2498.0,Convention Center / 4th St. @ MetroRail
67149,6,6,2014,18:12,1,NaN,2542.0,Plaza Saltillo,2542.0,Plaza Saltillo
53575,5,10,2014,12:12,1,NaN,2550.0,Republic Square @ Guadalupe & 4th St.,2550.0,Republic Square @ Guadalupe & 4th St.
137504,11,7,2014,12:12,1,NaN,2712.0,Toomey Rd @ South Lamar,2712.0,Toomey Rd @ South Lamar
140544,11,12,2014,10:12,1,NaN,2542.0,Plaza Saltillo,2542.0,Plaza Saltillo
...,...,...,...,...,...,...,...,...,...,...
11511,3,8,2014,10:12,360,NaN,2498.0,Convention Center / 4th St. @ MetroRail,NaN,Repair Shop
190354,3,21,2015,23:12,410,NaN,2498.0,Convention Center / 4th St. @ MetroRail,2562.0,San Jacinto & 8th Street
35993,4,4,2014,20:12,757,NaN,2499.0,City Hall / Lavaca & 2nd,2499.0,City Hall / Lavaca & 2nd
186828,3,19,2015,18:12,847,NaN,2499.0,City Hall / Lavaca & 2nd,2550.0,Republic Square @ Guadalupe & 4th St.


In [346]:
# reassign trips DF by filtering out the NA values
trips = trips[trips['bikeid'].isna() == False]

---
#### Investigating Missing `start_station_id` Data

In [347]:
trips[trips['start_station_id'].isna() == True]

,month,day,year,start_time,duration_minutes,bikeid,start_station_id,start_station_name,end_station_id,end_station_name
835,1,9,2014,14:12,29,460.0,NaN,Main Office,NaN,Main Office
1443,1,14,2014,15:12,6,185.0,NaN,Main Office,2498.0,Convention Center / 4th St. @ MetroRail
2567,1,22,2014,17:12,14,975.0,NaN,Main Office,NaN,Main Office
3040,1,27,2014,7:12,1,975.0,NaN,Main Office,NaN,Main Office
3093,1,27,2014,17:12,12,370.0,NaN,Main Office,2499.0,City Hall / Lavaca & 2nd
...,...,...,...,...,...,...,...,...,...,...
319827,12,4,2015,13:12,4,981.0,NaN,Main Office,2569.0,East 11th St. & San Marcos
321760,12,9,2015,10:12,11,209.0,NaN,Main Office,2567.0,Palmer Auditorium
321867,12,9,2015,14:12,25,124.0,NaN,Main Office,2569.0,East 11th St. & San Marcos
322139,12,10,2015,11:12,5,273.0,NaN,Main Office,2569.0,East 11th St. & San Marcos


This is a lot of missing data, however, it might help if we specifically take a look at the `start_station_name`s to see what the NaN IDs belong to.

In [348]:
# print statements
print(f"Unique station names with NaN ID: {len(trips[trips['start_station_id'].isna() == True]['start_station_name'].value_counts())}")
print("="*45)

# station NAMES w/ NaN IDs
trips[trips['start_station_id'].isna() == True]['start_station_name'].value_counts()

Unique station names with NaN ID: 18


start_station_name
Zilker Park at Barton Springs & William Barton Drive    11242
ACC - West & 12th                                        2388
Mobile Station                                           1107
East 11th Street at Victory Grill                        1014
Red River @ LBJ Library                                   560
Mobile Station @ Bike Fest                                485
Main Office                                               277
MapJam at Pan Am Park                                      32
MapJam at French Legation                                  26
MapJam at Hops & Grain Brewery                             19
Repair Shop                                                12
MapJam at Scoot Inn                                        11
Mobile Station @ Boardwalk Opening Ceremony                 9
Shop                                                        8
Re-branding                                                 5
Customer Service                                   

A lot of these station names are ones where there was likely no trip involved, especially ones named "Re-branding" or "Customer Service." The others are not frequently used. Perhaps the "MapJam" stations were pop-up stations set up by the bike-share company during specific events. Moreover, some of these names do not provide a geographical reference for us to fact check if the trip took palce, since we do not know what or ***where*** 'Mobile Station' could be. Therefore, I will go through these names and drop a majority of them that do not provide any further information for us to recognize it as a station. Specifically, I will keep:
- Zilker Park at Barton Springs & William Barton Drive
- ACC - West & 12th
- East 11th Street at Victory Grill
- Red River @ LBJ Library

In [349]:
# assign names to be kept
keep_station_names = ['Zilker Park at Barton Springs & William Barton Drive', 
                            'ACC - West & 12th', 
                            'East 11th Street at Victory Grill', 
                            'Red River @ LBJ Library']

# storing NAMES w/ NaN IDs
start_station_nans = list(trips[trips['start_station_id'].isna() == True]['start_station_name'].value_counts().index)

In [350]:
# creating drop list
drop_start_station_names = []

for start_name in start_station_nans:
    if start_name not in keep_station_names:
        drop_start_station_names.append(start_name)

drop_start_station_names

['Mobile Station',
 'Mobile Station @ Bike Fest',
 'Main Office',
 'MapJam at Pan Am Park',
 'MapJam at French Legation',
 'MapJam at Hops & Grain Brewery',
 'Repair Shop',
 'MapJam at Scoot Inn',
 'Mobile Station @ Boardwalk Opening Ceremony',
 'Shop',
 'Re-branding',
 'Customer Service',
 'Mobile Station @ Unplugged',
 'Marketing Event']

In [351]:
# want to drop these rows
trips[trips['start_station_name'].isin(drop_start_station_names)].shape

(2003, 10)

In [352]:
# reassign trips on the inverse of the above, drops 2003 rows
trips = trips[~trips['start_station_name'].isin(drop_start_station_names)]

trips.shape

(319756, 10)

---
#### Investigating Missing `end_station_id` Data

In [353]:
trips[trips['end_station_id'].isna() == True]

,month,day,year,start_time,duration_minutes,bikeid,start_station_id,start_station_name,end_station_id,end_station_name
1236,1,12,2014,11:12,1587,429.0,2500.0,Republic Square,NaN,Repair Shop
1514,1,15,2014,10:12,12,425.0,2499.0,City Hall / Lavaca & 2nd,NaN,Repair Shop
2268,1,20,2014,7:12,1767,479.0,2502.0,Barton Springs & Riverside,NaN,Shop
2727,1,25,2014,15:12,11,446.0,2496.0,8th & Congress,NaN,Repair Shop
2824,1,26,2014,3:12,813,320.0,2498.0,Convention Center / 4th St. @ MetroRail,NaN,Repair Shop
...,...,...,...,...,...,...,...,...,...,...
327267,12,26,2015,16:12,42,100.0,2572.0,Barton Springs Pool,NaN,Customer Service
327268,12,26,2015,16:12,41,226.0,2572.0,Barton Springs Pool,NaN,Customer Service
327444,12,28,2015,14:12,57,226.0,2567.0,Palmer Auditorium,NaN,Main Office
328037,12,30,2015,14:12,6,588.0,2499.0,City Hall / Lavaca & 2nd,NaN,Customer Service


Similar workflow to `start_station_id` missing values, I'll look at the station names.

In [354]:
# print statements
print(f"Unique end station names with NaN ID: {len(trips[trips['end_station_id'].isna() == True]['end_station_name'].value_counts())}")
print("="*45)

trips[trips['end_station_id'].isna() == True]['end_station_name'].value_counts()

# looks like it's a lot of the same names as the start stations

Unique end station names with NaN ID: 20


end_station_name
Zilker Park at Barton Springs & William Barton Drive    12783
ACC - West & 12th                                        2122
Mobile Station                                            783
East 11th Street at Victory Grill                         743
Red River @ LBJ Library                                   517
Main Office                                               271
Repair Shop                                               131
Customer Service                                          131
Mobile Station @ Bike Fest                                 95
Shop                                                       81
Missing                                                    36
MapJam at Pan Am Park                                      21
MapJam at Hops & Grain Brewery                             20
MapJam at French Legation                                  17
Mobile Station @ Unplugged                                 13
Marketing Event                                      

Similarly, a lot of the end stations have odd names. We can drop these values since, most likely, they were not trips. I will select from 'main Office' downward and drop those. Many of them also have much less traffic than what we might expect in 2 years.

In [355]:
# create list to drop
drop_end_station_names = []

# list of end station names with NaN
end_station_nans = list(trips[trips['end_station_id'].isna() == True]['end_station_name'].value_counts().index)

In [356]:
# use same var for keeping station names as in the start_station workflow
for end_name in end_station_nans:
    if end_name not in keep_station_names:
        drop_end_station_names.append(end_name)
        
drop_end_station_names

['Mobile Station',
 'Main Office',
 'Repair Shop',
 'Customer Service',
 'Mobile Station @ Bike Fest',
 'Shop',
 'Missing',
 'MapJam at Pan Am Park',
 'MapJam at Hops & Grain Brewery',
 'MapJam at French Legation',
 'Mobile Station @ Unplugged',
 'Marketing Event',
 'Stolen',
 'MapJam at Scoot Inn',
 'Mobile Station @ Boardwalk Opening Ceremony',
 'Main Shop']

In [357]:
# want to drop these rows
trips[trips['end_station_name'].isin(drop_end_station_names)].shape

(1631, 10)

In [358]:
# reassign trips DF on the inverse of the above
trips = trips[~trips['end_station_name'].isin(drop_end_station_names)]

trips.shape

(318125, 10)

---
### 1c. Fixing Columns

Now that I have gone through the data and taken care of potentially troublesome data via rows, we can look to fix columns. <br><br> First, I can drop variables (`end_station_id`, `end_station_name`, `bikeid`) that likely won't assist me in the modeling process or give me relevant EDA for my exact prompt (forecasting ***daily*** revenue for start stations). Additionally, since I have already examined rows affected by specific values of  `end_station_id` and name, I should be OK to remove these columns to tidy up the data a little more.

In [359]:
# dropping end_station_id, end_station_name
trips.drop(['end_station_name', 'end_station_id', 'bikeid'], axis = 1, inplace = True)
trips.head()

,month,day,year,start_time,duration_minutes,start_station_id,start_station_name
0,1,1,2014,0:12,20,2495.0,4th & Congress
1,1,1,2014,0:12,19,2495.0,4th & Congress
2,1,1,2014,0:12,17,2495.0,4th & Congress
3,1,1,2014,0:12,8,2498.0,Convention Center / 4th St. @ MetroRail
4,1,1,2014,2:12,15,2496.0,8th & Congress


In [360]:
# looking at data types
trips.dtypes

month                   int64
day                     int64
year                    int64
start_time             object
duration_minutes        int64
start_station_id      float64
start_station_name     object
dtype: object

I want to edit the data types so that ID is not a float (or integer), but rather an object just for cleanliness purposes. 
Converting `start_station_id` will be tricky since there are NaN values which default to 'float'. To fix this, and to ease the aggregation by `start_station_id` later, I will assign randomized ***unique*** IDs to the remaining station names that have NaN IDs.

In [361]:
# create list of existing unique ids
existing_ids = list(trips['start_station_id'].unique())
existing_ids.sort()
existing_ids # preview structure of IDs

[1002.0,
 1003.0,
 1005.0,
 1006.0,
 1007.0,
 1008.0,
 2494.0,
 2495.0,
 2496.0,
 2497.0,
 2498.0,
 2499.0,
 2500.0,
 2501.0,
 2502.0,
 2503.0,
 2504.0,
 2536.0,
 2537.0,
 2538.0,
 2540.0,
 nan,
 2541.0,
 2542.0,
 2544.0,
 2545.0,
 2546.0,
 2547.0,
 2548.0,
 2549.0,
 2550.0,
 2552.0,
 2561.0,
 2562.0,
 2563.0,
 2564.0,
 2565.0,
 2566.0,
 2567.0,
 2568.0,
 2569.0,
 2570.0,
 2571.0,
 2572.0,
 2574.0,
 2575.0,
 2576.0,
 2707.0,
 2711.0,
 2712.0,
 2822.0,
 2823.0,
 3291.0,
 3292.0,
 3293.0,
 3377.0,
 3381.0]

In [428]:
# we kept these station names, but they still have NaN IDs
keep_station_names

['Zilker Park at Barton Springs & William Barton Drive',
 'ACC - West & 12th',
 'East 11th Street at Victory Grill',
 'Red River @ LBJ Library']

In [365]:
# now, let's create 4 glaringly obvious IDs for our 4 station names that do not have an associated ID

new_ids = [9991, 9992, 9993, 9994]

new_nan_ids = dict(zip(keep_station_names, new_ids)) # zip them together into key, value pairs

# can now use this dictionary to replace the NaN values by station_name in the dataframe --> we can unique identify these for aggregation later!
new_nan_ids

{'Zilker Park at Barton Springs & William Barton Drive': 9991,
 'ACC - West & 12th': 9992,
 'East 11th Street at Victory Grill': 9993,
 'Red River @ LBJ Library': 9994}

In [366]:
# for each unique name, replace NaN in station_id column with the custom ID
for k, v in new_nan_ids.items():
    trips.loc[trips['start_station_name'] == f'{k}', 'start_station_id'] = v

In [369]:
# testing to see if it runs properly
trips[trips['start_station_name'].isin(keep_station_names)].sample(10)

,month,day,year,start_time,duration_minutes,start_station_id,start_station_name
138591,11,8,2014,16:12,25,9994.0,Red River @ LBJ Library
152860,1,3,2015,15:12,43,9991.0,Zilker Park at Barton Springs & William Barton...
171160,3,8,2015,18:12,14,9991.0,Zilker Park at Barton Springs & William Barton...
112542,9,28,2014,18:12,23,9991.0,Zilker Park at Barton Springs & William Barton...
31526,3,25,2014,20:12,23,9993.0,East 11th Street at Victory Grill
46316,4,25,2014,16:12,52,9992.0,ACC - West & 12th
119102,10,7,2014,7:12,5,9992.0,ACC - West & 12th
138111,11,8,2014,11:12,29,9991.0,Zilker Park at Barton Springs & William Barton...
56944,5,17,2014,16:12,25,9991.0,Zilker Park at Barton Springs & William Barton...
138596,11,8,2014,16:12,11,9991.0,Zilker Park at Barton Springs & William Barton...


In [370]:
# there should be no more NaN IDs & no more missing data overall!
trips.isna().sum()

month                 0
day                   0
year                  0
start_time            0
duration_minutes      0
start_station_id      0
start_station_name    0
dtype: int64

In [371]:
# convert start_station_id to int (remove decimal), then object
trips['start_station_id'] = trips['start_station_id'].astype(int).astype(object)


In [420]:
# let's add a single date and revenue column
trips['trip_revenue'] = (1.00 + 0.10*trips['duration_minutes']).round(2)

# date
trips['date'] = pd.to_datetime(trips[['month', 'day', 'year']])
trips['date'] = trips['date'].dt.strftime('%-m/%-d/%y')

One final consideration I want to make is for `start_time`. I believe this could prove to be ***extremely*** valuable at a granular level to predict demand (or count of rides) on an hourly basis, and we could further use this data to predict demand. However, for now, I am going to exlude `start_time` so I can aggregate the data by station ID, and keep it in a separately saved dataframe. Since the prompt is requesting predictions of daily revenue **by** station, the data fed to the model will not be granular by trip, but instead aggregated for all trips by station per day.

In [421]:
# preview of the unaggregated data I'm saving
trips.head()

,month,day,year,duration_minutes,start_station_id,start_station_name,trip_revenue,date
0,1,1,2014,20,2495,4th & Congress,3.0,1/1/14
1,1,1,2014,19,2495,4th & Congress,2.9,1/1/14
2,1,1,2014,17,2495,4th & Congress,2.7,1/1/14
3,1,1,2014,8,2498,Convention Center / 4th St. @ MetroRail,1.8,1/1/14
4,1,1,2014,15,2496,8th & Congress,2.5,1/1/14


In [374]:
# save 1st iteration of 'trips' with start_time and unaggregated
trips.to_csv("./cleaned_data/trips_cleaned_unaggregated.csv", index = False)

### 1d. Aggregating the Data

Finally, let's aggregate the data. This will involve me grouping by date (month, day, year) and station ID and name. I will sum up the duration and trip revenue for each station daily.

In [375]:
# drop start_time
trips.drop(['start_time'], axis = 1, inplace = True)
trips.head()

,month,day,year,duration_minutes,start_station_id,start_station_name,trip_revenue,date
0,1,1,2014,20,2495,4th & Congress,3.0,1/1/14
1,1,1,2014,19,2495,4th & Congress,2.9,1/1/14
2,1,1,2014,17,2495,4th & Congress,2.7,1/1/14
3,1,1,2014,8,2498,Convention Center / 4th St. @ MetroRail,1.8,1/1/14
4,1,1,2014,15,2496,8th & Congress,2.5,1/1/14


In [385]:
# quick assertion check to ensure number of unique IDs == number of unique Names
assert(len(trips['start_station_id'].unique()) == len(trips['start_station_name'].unique()))

In [386]:
# store columns we want to groupby
groupby_cols = ['start_station_id', 'start_station_name', 'month', 'day', 'year', 'date']

# groupby month, day, year, date, id, name & sum on duration, revenue
trips_agg = trips.groupby(groupby_cols)[['duration_minutes','trip_revenue']].sum().reset_index().sort_values(by = ['year','month','day'], ascending = True)
trips_agg

,start_station_id,start_station_name,month,day,year,date,duration_minutes,trip_revenue
783,2494,2nd & Congress,1,1,2014,1/1/14,979,115.9
1503,2495,4th & Congress,1,1,2014,1/1/14,729,94.9
2228,2496,8th & Congress,1,1,2014,1/1/14,1056,129.6
2940,2497,Capitol Station / Congress & 11th,1,1,2014,1/1/14,1098,135.8
3655,2498,Convention Center / 4th St. @ MetroRail,1,1,2014,1/1/14,653,82.3
...,...,...,...,...,...,...,...,...
26966,3291,11th & San Jacinto,12,31,2015,12/31/15,10,2.0
27088,3292,East 4th & Chicon,12,31,2015,12/31/15,54,9.4
27209,3293,East 2nd & Pedernales,12,31,2015,12/31/15,28,4.8
27221,3377,MoPac Pedestrian Bridge @ Veterans Drive,12,31,2015,12/31/15,295,40.5


I want to add one additional column to get the ***TOTAL*** number of trips from a start station per day. This may assist in some EDA later on.

In [387]:
# now, I want the number of trips
count_of_trips = trips.groupby(groupby_cols)['duration_minutes'].count().reset_index()

count_of_trips.rename(columns = {'duration_minutes': 'trip_count'}, inplace = True)

In [388]:
# merge trip count with aggregated trips data, but just the 'trip_count' column
trips_agg = pd.merge(trips_agg, count_of_trips, on = groupby_cols, how = 'left')
trips_agg

,start_station_id,start_station_name,month,day,year,date,duration_minutes,trip_revenue,trip_count
0,2494,2nd & Congress,1,1,2014,1/1/14,979,115.9,18
1,2495,4th & Congress,1,1,2014,1/1/14,729,94.9,22
2,2496,8th & Congress,1,1,2014,1/1/14,1056,129.6,24
3,2497,Capitol Station / Congress & 11th,1,1,2014,1/1/14,1098,135.8,26
4,2498,Convention Center / 4th St. @ MetroRail,1,1,2014,1/1/14,653,82.3,17
...,...,...,...,...,...,...,...,...,...
28633,3291,11th & San Jacinto,12,31,2015,12/31/15,10,2.0,1
28634,3292,East 4th & Chicon,12,31,2015,12/31/15,54,9.4,4
28635,3293,East 2nd & Pedernales,12,31,2015,12/31/15,28,4.8,2
28636,3377,MoPac Pedestrian Bridge @ Veterans Drive,12,31,2015,12/31/15,295,40.5,11


Since `trip_revenue` per day is now a function of both `trip_count` and `duration_minutes`, we can verify the data is logical. <br> 
<center>$revenue = 1*N_{\text{trips}} + 0.10*minutes$

Finally, I want to add a variable that represents the day of the week. This could prove to be a useful variable for predictive purposes.

In [426]:
# add day of week associated with the date
trips_agg['day_of_week'] = pd.to_datetime(trips_agg[['year','month','day']]).dt.day_name()

trips_agg.sample(10) # preview

,start_station_id,start_station_name,month,day,year,date,duration_minutes,trip_revenue,trip_count,day_of_week
21444,2541,State Capitol @ 14th & Colorado,7,27,2015,7/27/15,26,3.6,1,Monday
13740,2536,Waller & 6th St.,1,24,2015,1/24/15,116,16.6,5,Saturday
3378,2496,8th & Congress,5,3,2014,5/3/14,949,108.9,14,Saturday
15996,2822,East 6th at Robert Martinez,3,23,2015,3/23/15,89,14.9,6,Monday
24136,2571,Red River & 8th Street,9,23,2015,9/23/15,132,19.2,6,Wednesday
12983,2541,State Capitol @ 14th & Colorado,1,2,2015,1/2/15,8,1.8,1,Friday
21109,2567,Palmer Auditorium,7,19,2015,7/19/15,184,27.4,9,Sunday
22907,3293,East 2nd & Pedernales,8,28,2015,8/28/15,40,7.0,3,Friday
3479,2564,5th & San Marcos,5,5,2014,5/5/14,38,7.8,4,Monday
26098,2561,State Capitol Visitors Garage @ San Jacinto & ...,11,4,2015,11/4/15,34,4.4,1,Wednesday


In [464]:
# rearranging columns --> moving date to the front
date_col = trips_agg.pop('date')
trips_agg.insert(0, 'date', date_col)
trips_agg.head(3)

,date,start_station_id,start_station_name,month,day,year,duration_minutes,trip_revenue,trip_count,day_of_week
0,1/1/14,2494,2nd & Congress,1,1,2014,979,115.9,18,Wednesday
1,1/1/14,2495,4th & Congress,1,1,2014,729,94.9,22,Wednesday
2,1/1/14,2496,8th & Congress,1,1,2014,1056,129.6,24,Wednesday


---
---
---
### 1e. Fixing the Time Series (back after revelation in Notebook IV)

In [429]:
# sample preview for '2nd & Congress'
trips_agg[trips_agg['start_station_name']=='2nd & Congress']

## there is a gap in this time-series for 1/5/14

,start_station_id,start_station_name,month,day,year,date,duration_minutes,trip_revenue,trip_count,day_of_week
0,2494,2nd & Congress,1,1,2014,1/1/14,979,115.9,18,Wednesday
11,2494,2nd & Congress,1,2,2014,1/2/14,430,61.0,18,Thursday
22,2494,2nd & Congress,1,3,2014,1/3/14,14,3.4,2,Friday
33,2494,2nd & Congress,1,4,2014,1/4/14,82,9.2,1,Saturday
53,2494,2nd & Congress,1,6,2014,1/6/14,6,1.6,1,Monday
...,...,...,...,...,...,...,...,...,...,...
28399,2494,2nd & Congress,12,26,2015,12/26/15,164,28.4,12,Saturday
28461,2494,2nd & Congress,12,28,2015,12/28/15,17,5.7,4,Monday
28499,2494,2nd & Congress,12,29,2015,12/29/15,328,55.8,23,Tuesday
28547,2494,2nd & Congress,12,30,2015,12/30/15,534,71.4,18,Wednesday


While prepping to model, I filtered on my dataset for one specific station. I realized there were gaps in my time-series for specific stations. When I aggregated, if there were no trips for a specific day, there are no resulting records. This does not mean the data is missing. Instead, since there were just no trips on the associated day, the value should be present for the day but as 0. <br><br> For example, above I printed the first few days for "2nd & Congress," there is no row for 1/5/2014. The day is not missing, but instead there were just no trips taken that day. There should be a record there for this station, but with values of 0 for trip metrics. <br><br> To fix this so that we can get time-series data when filtering on stations, I need to ensure there is at least 1 day record for every single station regardless of whether there were trips or not. This needs to be fixed because to model this time-series, I plan to use ARIMA. ARIMA stands for Auto-Regressive (AR) Integrated (I) Moving Average (MA). If the time-series has gaps, especially ones that are not consistent, future predictions will be affected since ARIMA uses past data to predict future data.

In [591]:
# creating a new date range from scratch
dates = pd.DataFrame(pd.date_range(start = '2014-01-01', end = '2015-12-31', freq = 'D'))[0].dt.strftime('%-m/%-d/%y')
dates

0        1/1/14
1        1/2/14
2        1/3/14
3        1/4/14
4        1/5/14
         ...   
725    12/27/15
726    12/28/15
727    12/29/15
728    12/30/15
729    12/31/15
Name: 0, Length: 730, dtype: object

I thought it would be more efficient to do `dates = trips_agg['date'].unique()` to get the dates. However, I realized this solution would break in the off chance that there would be a missing day (i.e. all unique stations had NO trips, meaning there would be no date for that corresponding occurrence).

In [514]:
# all unique bike stations
bike_stations = trips_agg[['start_station_id', 'start_station_name']].drop_duplicates()

print(f"Number of Unique Stations: {len(bike_stations)}")
print("="*50)
bike_stations

Number of Unique Stations: 60


,start_station_id,start_station_name
0,2494,2nd & Congress
1,2495,4th & Congress
2,2496,8th & Congress
3,2497,Capitol Station / Congress & 11th
4,2498,Convention Center / 4th St. @ MetroRail
5,2499,City Hall / Lavaca & 2nd
6,2500,Republic Square
7,2501,5th & Bowie
8,2502,Barton Springs & Riverside
9,2503,South Congress & James


In [513]:
# combining EACH day (730) with ALL unq bike stations (60) --> 43,800 rows!
    # ts = time-series

# lots of unpacking here!
# ensure calling "df.values"
complete_ts = pd.DataFrame([(date, station_id, station_name) for date in dates for station_id, station_name in bike_stations.values], 
                           columns = ['date', 'start_station_id', 'start_station_name'])

# 43800 rows, this is correct
complete_ts

,date,start_station_id,start_station_name
0,1/1/14,2494,2nd & Congress
1,1/1/14,2495,4th & Congress
2,1/1/14,2496,8th & Congress
3,1/1/14,2497,Capitol Station / Congress & 11th
4,1/1/14,2498,Convention Center / 4th St. @ MetroRail
...,...,...,...
43795,12/31/15,2574,Zilker Park
43796,12/31/15,1006,Zilker Park West
43797,12/31/15,2546,ACC - West & 12th Street
43798,12/31/15,3377,MoPac Pedestrian Bridge @ Veterans Drive


In [555]:
# now, merge the completed time-series with the aggregated data
trips_agg_ts = pd.merge(complete_ts, trips_agg, on = ['date', 'start_station_id', 'start_station_name'], how = 'left')

In [556]:
# preview for '2nd & Congress' again
trips_agg_ts[trips_agg_ts['start_station_id'] == 2494]

,date,start_station_id,start_station_name,month,day,year,duration_minutes,trip_revenue,trip_count,day_of_week
0,1/1/14,2494,2nd & Congress,1.0,1.0,2014.0,979.0,115.9,18.0,Wednesday
60,1/2/14,2494,2nd & Congress,1.0,2.0,2014.0,430.0,61.0,18.0,Thursday
120,1/3/14,2494,2nd & Congress,1.0,3.0,2014.0,14.0,3.4,2.0,Friday
180,1/4/14,2494,2nd & Congress,1.0,4.0,2014.0,82.0,9.2,1.0,Saturday
240,1/5/14,2494,2nd & Congress,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
43500,12/27/15,2494,2nd & Congress,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43560,12/28/15,2494,2nd & Congress,12.0,28.0,2015.0,17.0,5.7,4.0,Monday
43620,12/29/15,2494,2nd & Congress,12.0,29.0,2015.0,328.0,55.8,23.0,Tuesday
43680,12/30/15,2494,2nd & Congress,12.0,30.0,2015.0,534.0,71.4,18.0,Wednesday


Now, the previous gap on 1/5/2014 is filled with a record! Because of the merge, the values pulled in show as NaNs. However, they are not missing data. We can fill these with 0s since, before aggregating, there were just no trip data for this station on that day. This is important information that needs to be known for the time-series for each station for forecasting purposes.

In [557]:
# replace the numeric column NaNs with 0
trips_agg_ts[['duration_minutes','trip_revenue','trip_count']] = trips_agg_ts[['duration_minutes','trip_revenue','trip_count']].fillna(0)

In [562]:
# reset month, day, year columns with corresponding values

# first convert 'date' to datetime
trips_agg_ts['date'] = pd.to_datetime(trips_agg_ts['date'], format = '%m/%d/%y')

trips_agg_ts['month'] = trips_agg_ts['date'].dt.month
trips_agg_ts['day'] = trips_agg_ts['date'].dt.day
trips_agg_ts['year'] = trips_agg_ts['date'].dt.year

In [564]:
# finally, let's reset the 'day_of_week'
trips_agg_ts['day_of_week'] = trips_agg_ts['date'].dt.day_name()

In [588]:
# let's also sort by start_station_id, this should give us a full panel dataset (60 unique stations w/ time-series)
trips_agg_ts.sort_values(by = ['start_station_id', 'date'], ascending = True, inplace = True)
trips_agg_ts.reset_index(drop = True)

,date,start_station_id,start_station_name,month,day,year,duration_minutes,trip_revenue,trip_count,day_of_week
0,2014-01-01,1002,6th & Navasota St.,1,1,2014,0.0,0.0,0.0,Wednesday
1,2014-01-02,1002,6th & Navasota St.,1,2,2014,0.0,0.0,0.0,Thursday
2,2014-01-03,1002,6th & Navasota St.,1,3,2014,0.0,0.0,0.0,Friday
3,2014-01-04,1002,6th & Navasota St.,1,4,2014,0.0,0.0,0.0,Saturday
4,2014-01-05,1002,6th & Navasota St.,1,5,2014,0.0,0.0,0.0,Sunday
...,...,...,...,...,...,...,...,...,...,...
43795,2015-12-27,9994,Red River @ LBJ Library,12,27,2015,0.0,0.0,0.0,Sunday
43796,2015-12-28,9994,Red River @ LBJ Library,12,28,2015,0.0,0.0,0.0,Monday
43797,2015-12-29,9994,Red River @ LBJ Library,12,29,2015,0.0,0.0,0.0,Tuesday
43798,2015-12-30,9994,Red River @ LBJ Library,12,30,2015,0.0,0.0,0.0,Wednesday


In [589]:
# final preview of '2nd & Congress'
trips_agg_ts[trips_agg_ts['start_station_id'] == 2494]

,date,start_station_id,start_station_name,month,day,year,duration_minutes,trip_revenue,trip_count,day_of_week
0,2014-01-01,2494,2nd & Congress,1,1,2014,979.0,115.9,18.0,Wednesday
60,2014-01-02,2494,2nd & Congress,1,2,2014,430.0,61.0,18.0,Thursday
120,2014-01-03,2494,2nd & Congress,1,3,2014,14.0,3.4,2.0,Friday
180,2014-01-04,2494,2nd & Congress,1,4,2014,82.0,9.2,1.0,Saturday
240,2014-01-05,2494,2nd & Congress,1,5,2014,0.0,0.0,0.0,Sunday
...,...,...,...,...,...,...,...,...,...,...
43500,2015-12-27,2494,2nd & Congress,12,27,2015,0.0,0.0,0.0,Sunday
43560,2015-12-28,2494,2nd & Congress,12,28,2015,17.0,5.7,4.0,Monday
43620,2015-12-29,2494,2nd & Congress,12,29,2015,328.0,55.8,23.0,Tuesday
43680,2015-12-30,2494,2nd & Congress,12,30,2015,534.0,71.4,18.0,Wednesday


The dataset is now fixed! Now, when I filter on a specific station and get a time-series returned, there should be no more missing days (rows) causing gaps in trends!

Now, we can save off the aggregated data! It is ready to be merged with weather data.

In [590]:
# save aggregated data
trips_agg_ts.to_csv('./cleaned_data/trips_cleaned_aggregated.csv', index = False)